In [1]:
from length_bayesian import BayesianSentenceLengthSkewModel

In [ ]:
# Example usage
import pandas as pd
df = pd.read_csv('../data/child_response_pairs_scored.csv')

df = df.drop_duplicates(subset=['text', 'CHI_response']).reset_index(drop=True)

#replace  \[\^ [a-z]*\] with ""
texts = df['CHI_response'].values.tolist()



# Create and fit model
model = BayesianSentenceLengthSkewModel()
model.fit(texts)
print(f"Median sentence length: {model.median_length}")

# Make predictions
test_texts = [
    "Short.",
    "another short one here.",
    "This is an average length sentence.",
    "This is a very long sentence with many more words than the average training data sldk d d fd ff d f."
]

for text in test_texts:
    score = model.predict(text,temperature=2)
    print(f"Text: '{text}' → Score: {score:.4f}")

# Save model
model.save("../models/bayesian_sentence_length_model.model")

Median sentence length: 5.0
Text: 'Short.' → Score: 0.6113
Text: 'another short one here.' → Score: 0.9817
Text: 'This is an average length sentence.' → Score: 0.9997
Text: 'This is a very long sentence with many more words than the average training data.' → Score: 0.9650


In [ ]:
from sentence_transformers import CrossEncoder
model_coherence = CrossEncoder("../models/child_coherence_model",device='cuda',max_length=356)#, automodel_args=automodel_args)
model_coherence2_2 = CrossEncoder("cross-encoder/stsb-roberta-large",device='cuda',max_length=356)#, automodel_args=automodel_args)
model_coherence2 = CrossEncoder("cross-encoder/quora-roberta-base",device='cuda',max_length=356)#, automodel_args=automodel_args)
model_coherence3 = CrossEncoder("cross-encoder/nli-deberta-v3-base")#use [:,1] after prediction

models = [model_coherence,model_coherence2,model_coherence3]

scores = []
coherent_examples = [
("What color is your blue hat?", "My hat is blue."),
("Do you want an apple?", "Yes, I want an apple."),
("Where is your toy car?", "The toy car is here."),
("How old are you?", "I am six."),
("Can you count to five?", "One two three four five."),
("Are you feeling happy?", "Yes, I feel happy."),
("What did you build today?", "I built a red tower."),
("Do you want to play now?", "I want to play."),
("Where is your book?", "My book is on the table."),
("Do you like ice cream?", "I love ice cream.")
]

random_responses = [
("Why does the clock hum?", "My umbrella sings softly."),
("Is the spoon a spaceship?", "My pencil dreams of rainbows."),
("Can you ride the sound?", "The book flew away."),
("Where is the silent banana?", "My shoes are dancing slow."),
("Do you think the wind eats?", "Candy falls from sky."),
("Will the number smile today?", "The jelly is laughing loudly."),
("Does the river taste bread?", "My water sings in color."),
("Is the car playing chess?", "The mirror smiles absurdly."),
("Can chairs whisper secrets?", "Balloons argue with trees."),
("Where does the shadow travel?", "The cheese forgets its tune."),
("Why is the sky wearing boots?", "Cats number the empty stars."),
("Do apples sing to moon?", "The clock jumps over apples."),
("Is time a slippery pancake?", "My cookie runs on wheels."),
("Can dreams fill the box?", "The rain whispers to paper."),
("Do pencils converse softly?", "My brother sings underwater."),
("Will the chair jump tonight?", "The fruit dreams of dancing."),
("Why do letters feel cold?", "The ball giggles at silence."),
("Is the table floating in perfume?", "My phone thinks on elephants."),
("Can the pancake swim today?", "The radio is a runaway kite."),
("Does music grow on flowers?", "My sandwich reads a mystery.")
]

hard_random_examples = [
("What color is the ball?", "The ball is on the red bench."),
("Where is your toy?", "My toy is beside the sunny porch."),
("Do you want to play?", "Yes, but the carpet is slippery."),
("Are you ready for lunch?", "Lunch is on the blue sofa."),
("Can you read a story?", "I read a story about floating chairs."),
("What did you see outside?", "I saw a rainbow near the tall clock."),
("Where is your shoe?", "The shoe jumped off the apple tree."),
("Do you like the game?", "The game raced past the little boat."),
("Will you wear your jacket?", "My jacket is resting at the blue fountain."),
("Are we going home?", "Home is next to the green mountain."),
]
repeated_hard_random_examples = [
("What color is the ball?", "What color is the ball?"),
("How old are you?", "How old are you?"),
("Can you count to five?", "Can you count to five?"),
("Where is your toy?", "Where is your toy?"),
("Do you want to play now?", "Do you want to play now?"),
("What did you build today?", "What did you build today?"),
("Are you feeling happy?", "Are you feeling happy?"),
("Can you read a story?", "Can you read a story?"),
("Will you wear your jacket?", "Will you wear your jacket?"),
("Are we going home?", "Are we going home?"),
("What color is the ball?", "What color do you think is the ball?"),
("How old are you?", "How old do you think you are?"),
("Can you count to five?", "Can you count from one to five?"),
("Where is your toy?", "Your toy is where you left it."),
("Do you want to play now?", "Do you want to play with me now?"),
("What did you build today?", "Did you build something today."),
("Are you feeling happy?", "Are you feeling happy today?"),
("Can you read a story?", "Can you read a story to me?"),
("Will you wear your jacket?", "Will you wear your jacket today?"),
("Are we going home?", "Are we going home soon?")
]


sentances = coherent_examples + random_responses + hard_random_examples + repeated_hard_random_examples
sentances_reversed = [(s[1],s[0]) for s in sentances]
#take average of all 3 scores
score_1 = model_coherence.predict(sentances)#coherence
score_2 = model_coherence2.predict(sentances)#similarity
score_3 = model_coherence2_2.predict(sentances)#question similarity
score_ = model_coherence3.predict(sentances)# coherence logic
score_4 = score_[:,0]/5
score_5 = score_[:,1]/5
score_6 = score_[:,2]/5

final_scores = []
import math
def logistic(x, midpoint, steepness):
    """Return logistic function value given midpoint and steepness."""
    return 1 / (1 + math.exp(-steepness * (x - midpoint)))

for i in range(len(sentances)):
    # Extract for clarity
    s1   = score_1[i]
    s2   = score_2[i]
    s3  = score_3[i]
    s4 = score_4[i]
    s5 = score_5[i]
    s6 = score_6[i]
    #part to modify
    # 1) Reward moderate to high s1 (coherence). Let’s just keep it as-is:
    t2 = 3.0 * (s2 - s2**2)
    t3 = 3.0 * (s3 - s3**2)   # previously 4.0
    t6 = 2.0 * (s6 - s6**2)
    t1 = 1.0 * s1            # linear in s1

    # ——————————————————————————————
    # B) Penalties on “extremes”
    # ——————————————————————————————
    penalty_s4 = -1.0 * (s4**2)
    penalty_s5 = -0.5 * (s5**2)

    raw = t1 + t2 + t3 + t6 + penalty_s4 + penalty_s5

    # ——————————————————————————————
    # C) Additional shape rules
    # ——————————————————————————————
    # 1) If s3 is over 0.7, penalize further:
    if s3 > 0.7:
        raw -= 2.5 * (s3 - 0.7)**2   # bumped from 2.0 up to 2.5

    # 2) If both s2 and s3 are above 0.75, that often means “repeated,”
    #    so apply a compact penalty:
    if (s2 > 0.75) and (s3 > 0.75):
        raw -= 0.8

    # 3) Keep the existing s4 checks for non‐coherence:
    if s4 > 0.5:
        raw -= 2.0 * (s4 - 0.5)
    if s4 < -0.5:
        raw -= 1.5 * (-0.5 - s4)

    # ——————————————————————————————
    # D) Final scaling
    # ——————————————————————————————
    # Shift raw up by +2 and divide by 5, so “good” examples
    # tend to be in [0.6..0.8+] and “bad repeated” or “non‐coherent”
    # push further down.
    avg = (raw + 2.0) / 5.0

    #clipped = max(0, min(1, raw))
    final_scores.append(avg)
    if i == 0:
        print(f"Coherent Examples")
    if i == len(coherent_examples):
        print(f"Bad Non-Coherent Examples")
    if i == len(coherent_examples) + len(random_responses):
        print(f"Bad Coherent no-context Bad Examples")
    if i == len(coherent_examples) + len(random_responses) + len(hard_random_examples):
        print(f"Bad Coherent repeated Bad Examples")
    #print(f"Text: '{sentances[i][0]}' → '{sentances[i][1]}' → Score1: {score_1[i]:.4f} Score2: {score_2[i]:.4f} Score2_2: {score_2_2[i]:.4f} Score3: {score_3[i]:.4f} → Avg: {avg:.4f}")
    #print(f"Text: '{sentances[i][0]}' → '{sentances[i][1]}'")
    print(f"Score1: {score_1[i]:.4f}\tScore2: {score_2[i]:.4f}\tScore3: {score_3[i]:.4f}\tScore4: {score_4[i]:.4f}\tScore5: {score_5[i]:.4f}\tScore6: {score_6[i]:.4f} → Avg: {avg:.4f}")

Coherent Examples
Score1: 0.7649	Score2: 0.0052	Score3: 0.4563	Score4: -0.3843	Score5: -0.2534	Score6: 0.5937 → Avg: 0.8151
Score1: 0.7988	Score2: 0.8243	Score3: 0.5032	Score4: -0.3394	Score5: -0.5773	Score6: 0.9376 → Avg: 0.8137
Score1: 0.7496	Score2: 0.0073	Score3: 0.4593	Score4: 0.9339	Score5: -0.6467	Score6: -0.1614 → Avg: 0.2881
Score1: 0.8202	Score2: 0.0024	Score3: 0.0635	Score4: -0.3245	Score5: -0.6100	Score6: 0.9896 → Avg: 0.5589
Score1: 0.7997	Score2: 0.0101	Score3: 0.4880	Score4: 0.1541	Score5: -0.0321	Score6: -0.1436 → Avg: 0.6953
Score1: 0.8127	Score2: 0.2658	Score3: 0.4110	Score4: -0.5621	Score5: -0.5026	Score6: 1.0536 → Avg: 0.7436
Score1: 0.7565	Score2: 0.0027	Score3: 0.2329	Score4: -0.1727	Score5: -0.6176	Score6: 0.8502 → Avg: 0.7027
Score1: 0.7501	Score2: 0.0626	Score3: 0.4710	Score4: -0.6261	Score5: -0.2497	Score6: 0.8062 → Avg: 0.7246
Score1: 0.8107	Score2: 0.0022	Score3: 0.3055	Score4: 0.7512	Score5: -0.9373	Score6: 0.4210 → Avg: 0.5295
Score1: 0.7881	Score2: 0.2249

In [40]:
i = .99
(i - (i**2))*4

0.03960000000000008

In [43]:
#take min of Score2 and Score2_2
#if Score 3 is high and Score2 and Score2_2 is also high that is a bad call



model_coherence2.predict(("Can you count to five?","How old are you?"))

np.float32(0.0020019182)

In [2]:
score_1

array([0.79409254, 0.7756106 , 0.82671916, 0.836703  ], dtype=float32)

In [25]:
df = pd.read_csv('../data/child_response_pairs.csv')
df["mixed"] = df["child_coherence_score_prediction"] * 0.7 + predictions * 0.3

In [26]:
df_display = df.sort_values(by=['mixed'], ascending=False).head(10)

for index, row in df_display.iterrows():
    prediction = row[f"mixed"]
    print(f"{row['text']}-->> \t{row['CHI_response']}-->>\t{prediction}")

well, I put it in the garbage.-->> 	I put it in the garbage.-->>	0.8724676206391143
yeah. you threw away the catalogue that Gammy gave me.-->> 	you threw the catalogue that Gammy gave me.-->>	0.8721306117636413
that's daddy and Gus.-->> 	that is daddy and Gus.-->>	0.8720316762402344
I'm gonna get me some juice.-->> 	I'm gonna get me some juice.-->>	0.8682353681704712
I'm going to wash my hands.-->> 	I'm going to wash up my hands.-->>	0.8640208010774993
there's Baaee's thumb.-->> 	there is Baaee's thumb.-->>	0.863840229026413
that's a fishing boat.-->> 	a fishing boat.-->>	0.8633416848856353
the baby's shirt is all wrinkled.-->> 	baby's, shirt, all, wrinkled.-->>	0.8633239476805572
he has them on his head.-->> 	he have them on he head.-->>	0.8633134659323882
that's my comb.-->> 	that's my comb.-->>	0.863229350752716


In [1]:
from sentence_transformers import CrossEncoder
import pandas as pd
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
score_column = 'coherence_score'


#model = CrossEncoder('cross-encoder/stsb-roberta-large',max_length=96)
model = CrossEncoder("cross-encoder/nli-deberta-v3-base",device='cuda',max_length=96)
df = pd.read_csv('../data/child_response_pairs.csv')
texts = [[row['text'], row['CHI_response']] for _, row in df.iterrows()]


predictions = model.predict(texts, batch_size=1024, show_progress_bar=True)[:,1]
df[f"{score_column}_prediction"] = predictions
#df.to_csv('../data/child_response_pairs.csv', index=False)

#print top 10 predictions sorted by score
df_display = df.sort_values(by=f"{score_column}_prediction", ascending=False).head(15)


for index, row in df_display.iterrows():
    prediction = row[f"{score_column}_prediction"]
    print(f"{row['text']}-->> \t{row['CHI_response']}-->>\t{prediction}")

print('done')


/bsuhome/enochlevandovsky/miniforge3/envs/childes/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|██████████| 517/517 [09:12<00:00,  1.07s/it]


oh thank_you.-->> 	I'm gonna buy candy. I'm going to buy some candy.-->>	4.999378681182861
I see.-->> 	we're going to fed her after her nap. gonna feed her after her nap.-->>	4.999053001403809
I want carrot cake, Mom.-->> 	I would like carrot cake, Mommy.-->>	4.975674629211426
0are you sure?-->> 	I think I found something. I think I found.-->>	4.960228443145752
yeah.-->> 	I want go outside. I wanna go outside.-->>	4.960019111633301
okay you guys come here.-->> 	fix your poor dad. come and fix your poor daddy.-->>	4.955489635467529
she likes those though.-->> 	yeah, she does like toys. she does like toys though.-->>	4.953551292419434
he has no love for the Empire.-->> 	he has no love for the Empire.-->>	4.95037317276001
oh well.-->> 	he hadta he hadta. never he minds. I'll iron your stuff what do you think? I don't mind. I'd iron your.-->>	4.944336891174316
I got it.-->> 	I get it.-->>	4.942104339599609
oh.-->> 	Chim Chim Charlie loves karate Chim Chim Charlie love karate.-->>	4.9325299

In [4]:
predictions.max()

np.float32(4.9993787)

In [2]:
df[f"{score_column}_prediction"] = predictions
#df.to_csv('../data/child_response_pairs.csv', index=False)

#print top 10 predictions sorted by score
df_display = df.sort_values(by=f"{score_column}_prediction", ascending=False).head(15)


for index, row in df_display.iterrows():
    prediction = row[f"{score_column}_prediction"]
    print(f"{row['text']}-->> \t{row['CHI_response']}-->>\t{prediction}")

print('done')

okay.-->> 	so you move your pawn that is in front of your king up. no it is the pawn in front of your king up. and then next turn you take your bishop and put it about I think two two squares so that before it would hit the edge and then your queen all the way over to the edge. and then take your queen and go diagonal. go diagonal and it would it would take the pawn. and then your king would try and get your queen. but your bishop would be there. and it would it would take your king. so then so then you would be in checkmate.-->>	0.6587844491004944
so is there any other strategies for starting the game or?-->> 	well, if the person is like new to the game, the three move one. that is where you put the you move a pawn that is by their by the queen and another bishop. I think it's I am not sure what side it is on. but you move your bishop out so it is like diagonally from like one of the squares from where the queen is going to be. the queen moves out either diagonally. or it goes out dia

In [ ]:
model2 = BayesianSentenceLengthSkewModel.load("../models/bayesian_sentence_length_model.model")
model2.predict("This is a nice thing to do in summer.",temperature=4)

In [7]:
model2.predict("This is a nice thing to do in summer.",temperature=4)

0.32669073286908284

In [25]:
model2.predict("This is a very nice thing to do.",temperature=5)

0.6588692877481858

In [16]:
model.predict("This is a vce.",temperature=2)

0.9307539992989925

In [23]:
with open("../models/bayesian_sentence_length_model.model", 'r') as f:
    lines = f.readlines()
    for line in lines:
        if line.startswith("median_length="):
            median_length = float(line.split("=")[1])
            print(f"Median length: {median_length}")
            print(median_length == model.median_length)
        elif line.startswith("left_variance="):
            left_variance = float(line.split("=")[1])
            print(f"Left variance: {left_variance}")
            print(left_variance == model.left_variance)
        elif line.startswith("right_variance="):
            right_variance = float(line.split("=")[1])
            print(f"Right variance: {right_variance}")
            print(right_variance == model.right_variance)
        elif line.startswith("word_counts="):
            counts_str = line.split("=")[1]
            word_counts = [float(x) for x in counts_str.split(",") if x]
            print(f"Word counts: {word_counts}")
            print(word_counts == model.word_counts)

Median length: 2.0
True
Left variance: 0.6800962510446344
True
Right variance: 10.31034952228082
True
Word counts: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

In [17]:
model.right_variance

10.31034952228082

In [18]:
model.left_variance

0.6800962510446344

In [19]:
model.median_length

2.0

In [20]:
model.word_counts

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0